# Lineare Regression

In dieser Übung werden wir die in der Vorlesung kennengelernten Inhalte zur Regressionsanalyse direkt in R anwenden. Dafür werden wir uns zunächst eine einfache lineare Regression ansehen. Diese können wir mit Tools von `base` oder auch direkt in einer Grafik mit `ggplot2` umsetzen. 

Anschließend wollen wir die Decision Trees aus der vorhergehenden Übung mit der Regressionsanalyse verbinden. Wir werden uns ein Modell bauen, welches analog wie die klassifizierenden Decision Trees funktioniert, aber mit kontinuierlichen Variablen arbeiten kann! Wir erinnern uns – in der letzten Wochen hatten wir es mit Klassifizierungsproblemen zu tun. D.h., eine Zielvariable konnte nur eine begrenzte Anzahl an Kategorien annehmen (die Pilze konnten z.B. nur `edible` oder `poisonous` sein). Jetzt lernen wir ein Modell kennen, welches für numerische, kontinuierliche Variablen funktioniert.

Für die theoretischen Grundlagen der Regressionsanalyse sei an dieser Stelle wieder auf die Vorlesung verwiesen.

In [ ]:
pacman::p_load(
    tidyverse,
    tidymodels,
    vip,
    rpart.plot,
    palmerpenguins
    )

# Set the default plot width for Jupyter Notebook display
options(repr.plot.width = 12, repr.plot.height = 8, digits = 3)

## Lineare Regression

Wir erinnern uns an das Paket `palmerpenguins` aus der 2. Stunde? Über die Pinguine des Datensatzes wurden verschiedene Parameter erhoben, wie beispielsweise die Flügellänge `flipper_length_mm` und das Körpergewicht `body_mass_g`. Aus der Biologie ließe sich an dieser Stelle eventuell ein erster Zusammenhang zwischen Flüggellänge und Körpergewicht vermuten. Durch eine größere Flügellänge könnte man vielleicht auf einen größeren Körperbau und damit auch auf einen größeres Körpergewicht schließen. Eine gute Fragestellung für eine Regressionsanalyse. 

Wir vermuten erstmal einen linearen Zusammenhang zwischen Flüggellänge und Körpergewicht. Wie wir in der letzten Sitzung bereits kennengelernt haben, werden Datenanalyse-Modelle in R in der Regel durch eine Formel definiert, die eine Tilde `~` enthält. Die Tilde heißt soviel wie "wird vorhergesagt durch". Der Zusammenhang, den wir jetzt untersuchen, können wir also mit `flipper_length_mm ~ body_mass_g` definieren. Wir vermuten, dass das Körpergewicht linear von der Flügellänge abhängt. In `base` können wir lineare Modell mithilfe der Funktion `lm()` (für *linear model*) fitten. Wir übergeben `lm()` als erstes Argument unsere Formel `body_mass_g ~ flipper_length_mm` und an zweiter Stelle den Datensatz `penguins`. Anschließend lassen wir uns die Ergebnisse des Modells mithilfe von `summary()` ausgeben.

In [ ]:
lm(body_mass_g ~ flipper_length_mm, data = penguins) %>% 
    summary() 

Hui, da kommen erstmal viele Informationen auf einmal! Für uns sind zunächst einmal zwei Teile des Outputs interessant. Unter **Coefficients** können wir uns die Koeffizienten unseres Regressionsmodells ausgeben lassen. Unser Modell folgt der Formel $ \hat{y}_i = b_0 + b_1 * x_i $, wobei $\hat{y}$ die Vorhersage unserer Zielgröße `body_mass_g` und $x$ unsere Prädiktorvariable `flipper_length_mm` ist. Die anderen Parameter der Regressionsgleichung $b_0$ und $b_1$ unseres Modells werden in der Zusammenfassung unter Coefficients -> Estimate ausgegeben. Der Intercept ist $b_0$ (= vorhergesagter Wert für x = 0). Die Steigung der Regressionsgerade, also $b_1$ finden wird auch unter Estimates, in der Zeile neben der unabhängigen Variable `flipper_length_mm`. In dieser Form ist das ganze für uns natürlich eher schwierig interpretierbar, daher werden wir gleich kennenlernen, wie wir das lineare Modell visualisieren können.

Die zweite für uns interessante Variable ist **(Multiple) R-squared**. $R^2$, oder auch R-squared, drückt aus, wie viel der Varianz in der abhängigen Variable (hier `body_mass_g`) statistisch gesehen durch die Varianz in der unabhängigen erklärt wird (hier also `flipper_length_mm`). Mithilfe von R-squared können wir die Güte des Regressionsmodells bewerten. `0.759` ist zunächst mal ein ganz gutes R-squared, sodass wir von einem gewissen Zusammenhang zwischen `body_mass_g` und `flipper_length_mm` ausgehen können!

### In `tidymodels`

In `tidymodels` können wir diese Werte auch anders extrahieren, und zwar mit `tidy()` und `glance()`. Daher werden wir ab dieser Stelle mit dem `tidymodels`-Ansatz weitermachen.

In [ ]:
lm(body_mass_g ~ flipper_length_mm, data = penguins) %>% 
    tidy()

lm(body_mass_g ~ flipper_length_mm, data = penguins) %>% 
    glance()

### Visualisierung

Der Übersichtlichkeit halber wollen wir das lineare Modell jetzt einmal visualisieren. Dazu kombinieren zwir zwei Geome. Zunächst visualisieren wir den Zusammenhang zwischen $x$ & $y$ mit einer Punktewolke und `geom_point()`. Wir plotten `flipper_length_mm` auf der x-Achse und `body_mass_g` auf der y-Achse. Für die Regressionsgerade hat `ggplot2` praktischerweise ein eigenes **geom** eingebaut, und zwar `geom_smooth()`. Diesem Geom geben wir unsere Formel mit `y ~ x` (die Variablen haben wir im Mapping gewissermaßen "umgetauft"). 
Der Code dafür sieht dann folgendermaßen aus:

In [ ]:
penguins %>%
    drop_na() %>% # remove NA values
    ggplot(mapping = aes(x = flipper_length_mm, y = body_mass_g)) +
    geom_point(size = 2) +
    geom_smooth(method = "lm", formula = y ~ x) +
    theme_minimal(base_size = 20)

Wir können hier unseren vermuteten linearen Zusammenhang ganz gut erkennen, oder? Mit steigender Flüggellänge wird auch das Körpergewicht tendenziell schwerer. In der Realität muss man allerdings aufpassen, wo man welche Zusammenhänge vermuten kann. Wir haben in unserem Datensatz ja 3 verschiedene Spezies abgebildet, und es ist nicht automatisch gegeben, dass der Zusammenhang für alle diese Spezies gleich ist. Daher ist es sinnvoll, nochmal ein lineares Modell zu visualisieren, welches eine Regression getrennt je nach Spezies erstellt. Das funktioniert mit `ggplot2` ziemlich einfach, dafür müssen wir einfach `aes()` ein zusätzliches Argument übergeben.

1. Was für eine Grafik erwartet ihr, wenn die Regression für jede einzelne Spezies ausgeführt wird?

In [ ]:
penguins %>%
    drop_na() %>% # remove NA values
    ggplot(mapping = aes(x = flipper_length_mm, y = body_mass_g, color = species)) +
    geom_point(size = 2) +
    geom_smooth(method = "lm", formula = y ~ x) +
    theme_minimal(base_size = 20)

Voilà, wir bekommen jetzt einen Plot ausgegeben, bei dem wir pro Spezies ein eigenes lineares Modell abgebildet bekommen. Wenn wir uns wie weiter oben auch die genauen Koeffizienten und R-squared ausgeben lassen wollen, können wir das folgendermaßen machen.

2. Bevor ihr den Code ausführt, überlegt euch, was jede Zeile macht. Schlagt dafür gerne den Befehl `do()` einmal kurz nach!

In [ ]:
penguins %>%
    drop_na() %>%
    group_by(species) %>%
    do(tidy(lm(body_mass_g ~ flipper_length_mm, data = .)))

penguins %>%
    drop_na() %>%
    group_by(species) %>%
    do(glance(lm(body_mass_g ~ flipper_length_mm, data = .))) 

3. Wie würdet ihr die Werte für `r.squared` interpretieren?

> Natürlich können wir auch andere Zusammenhänge abbilden. Wir könnten als Formel auch `flipper_length_mm ~ body_mass_g` angeben, wenn wir ein umgekehretes Abhängikeitsverhältnis vermuten würden. Man könnte über `+` auch noch weitere Vorhersagevariablen in das Modell einbeziehen. Wie genau man ein Regressionsmodell spezifiziert ist vor allem eine Frage von theoretischen oder sachlogischen Überlegungen. Weitere Informationen zu den Möglichkeiten zur Umsetzung in R findet ihr wie gewohnt in der Dokumentation von `lm()`.

Wir werden in dieser Übung zunächst nicht weiter auf `lm()` eingehen, sondern möchten jetzt die Decision Trees von letzter Woche mit der Regressionsanalyse verbinden.

# Decision Tree Regression

Eine Regression mit Decision Trees funktioniert ähnlich wie eine Klassifikation, zielt aber darauf ab, statt diskreten Kategorien kontinuierliche Werte zu prognostizieren. Wir möchten wieder die Beziehung zwischen Prädiktor- und Zielvariable ermitteln. Der Decision Tree teilt den Datensatz entlang der Prädiktorvariablen in immer homgonere Teilgruppen auf, um möglichst genau die Beziehung zur Zielvariable zu bestimmen. Nachdem das Modell trainiert wurde, können wir es wieder zur Vorhersage neuer Datenpunkte verwenden! Decision Trees für die Regressionsanalyse haben den Vorteil, auch nichtlineare Beziehungen zwischen Prädiktoren und Zielvariable abbilden zu können! 

Wie letzte Woche werden wir zunächst ein Beispiel gemeinsam kennenlernen. Anschließend könnt ihr das erlernte Wissen nochmal auf ein weitere Beispiel anwenden.

## Bike sharing data

Als erstes Beispiel schauen wir uns einen Datensatz von Hadi Fanaee-T aus dem [UCI Machine Learning Repository](http://archive.ics.uci.edu/) zum Thema Bike Sharing an.

> This dataset contains the hourly and daily count of rental bikes between years 2011 and 2012 in Capital bikeshare system with the corresponding weather and seasonal information.

### Daten explorieren und vorbereiten

Der Datensatz liegt im Ordner `dw1/data/bike_sharing/`. Dort liegt auch eine Readme-Datei, in der die Dokumentation des Datensatzes zu finden ist.

4. Schaut euch die Readme-Datei des Datensatzes an und überlegt, welche Variablen von Interesse für eine genauere Untersuchung sein könnten.

Anschließend laden wir den Datensatz wie gewohnt. Uns interessieren zunächst einmal die stündlichen Aufnahmen, daher laden wir die Datei `hour.csv`. Die Variable `instant` ist der Index, daher schmeißen wir diesen raus.

In [ ]:
bikes = read_csv("data/bike_sharing/hour.csv", show_col_types = FALSE) %>%
    select(-instant) %>%
    glimpse()

Für eine Organisation, die Bike Sharing betreibt, ist vermutlich vor allem die Variable `cnt` interessant.

> cnt: count of total rental bikes including both casual and registered

Damit könnte eine Organisation beispielsweise eine Vorhersage darüber treffen, wieviele Fahrräder zu einem bestimmten Zeitpunkt gerade ausgeliehen sind. So könnte man beispielsweise besser planen zu welchen Zeiten besonders viele Fahrräder zur Verfügung stehen müssen. `cnt` ist in diesem Beispiel also unsere **Zielvariable**! Bevor wir ans Modellieren gehen, wollen wir aber wie letzte Woche die Variablen nochmals mithilfe von `facet_wrap()` darstellen.

In [ ]:
bikes %>%
    select(-dteday) %>%
    select(where(is.double)) %>%
    pivot_longer(cols = everything(), names_to = "variable", values_to = "value") %>%
    ggplot(aes(x = value)) +
        geom_histogram(bins = 30, color = "black", fill = "lightblue") +
        facet_wrap(~variable, scales = "free", ncol = 4) +
        labs(title = "Histograms of Numeric Variables in the bike sharing data set", x = "Value", y = "Frequency") +
        theme_minimal(base_size = 20)

#### Supervised Learning

<img src="https://rsample.tidymodels.org/logo.png" alt="rsample" width="100" align="right" /> In einem letzten Schritt der Vorbereitung teilen wir wie letzte Woche unseren Datensatz wieder in Trainings- und Testdaten auf. Die Variable `dteday` ist vom Typ `date` und daher können wir diese nicht in der Regressionsanalyse berücksichtigen (die Modelle funktionieren nur für numerische Prädiktoren). Abgesehen davon ist das Datum ja ebenfalls in den Variablen `yr` und `mnth` berücksichtigt.

In [ ]:
bikes_split <- bikes %>%
    select(-dteday) %>%
    select(-casual, -registered) %>%
    initial_split(prop = 0.75)

### Modell trainieren

<img src="https://recipes.tidymodels.org/logo.png" alt="recipe" width="100" align="right" /> Für das Modellieren gehen wir recht analog vor wie bei der Klassifikation mit Decision Trees. Allerdings wollen wir diese Woche das ganze noch etwas formalisieren. Im `tidymodels` Universum gibt es ein Paket, mit dem wir den Trainings- und Testprozess noch etwas Form geben können, und zwar das Paket `recipe`. Mit `recipe` können wir analog zu `dplyr` Pipe-Sequenzen von bestimmt Features des Modellierens erstellen. Dadurch können wir den ganzen Prozess etwas formalisieren und verallgemeinern. Außerdem beinhaltet `recipe` einige praktische Funktionen, die das Modell auch direkt noch verbessern.

Wir können für unser Beispiel direkt ein einfaches Rezept erstellen:

In [ ]:
bikes_recipe <- training(bikes_split) %>%
    recipe(cnt ~ .) %>%
    prep() %>%
    print()

Auch hier benutzen wir wieder die Tilde, um die Beziehung unserer Zielvariablen zu definieren. `cnt ~ .` bedeutet hier einfach, dass wir `cnt` als Zielvariable (outcome) und alle anderen Variablen als Prädiktoren (predictor) definieren! Wir setzen an dieser Stelle tatsächlich noch gar nicht fest, was wir eigentlich für ein Modell verwenden wollen (also ob z.B. lineare Regression oder ein Decision Tree). Außerdem gibt uns `recipe` auch direkt Informationen über den Umfang der Trainingsdaten.

In der Regel gibt es bei einem Datensatz mit so vielen Variablen auch Korrelationen unter den Prädiktoren. Das liegt gerade bei Wetterdaten auf der Hand - die `season` (Jahreszeit) hat z.B. vermutlich einen Einfluss auf die Temperatur `temp`. Stark korrelierende Prädiktoren stellen ein Problem für die Analyse dar. Man kann sie ausschließen, um die Genauigkeit des Modells zu verbessern. Dafür kennt `recipe` praktischerweise die Funktion `step_corr()`, welche wir mithilfe der Pipe einfach als zusätzlichen Schritt ins Rezept einbauen können.

In [ ]:
bikes_recipe <- training(bikes_split) %>%
    recipe(cnt ~ .) %>%
    step_corr(all_predictors()) %>%
    prep() %>%
    print()

Unser Rezept enthält jetzt auch noch den Abschnitt Operations. Wie wir sehen können filtert `recipe` automatisch die gefühlte Temperatur in Celsius `atemp` raus. Diese korreliert vermutlich mit der gemessenen Temperatur in Celsius `temp`, und zwar so stark, dass es besser ist,  sie für die Analyse ausschließen. 

Zwei weitere Operationen die sich bei kontinuierlichen numerischen Daten lohnen können, sind **Centering** und **Normalisation**, in Kombination auch **Standardisation** (oder z-standardisation) genannt. Dabei werden die Datenpunkte zentriert, sodass der Mittelwert jedes Prädiktors 0 ist, und anschließend normalisiert (genormt), sodass jeder Prädiktor eine Standardabweichung von 1 hat. Die Prädiktor-Variablen werden so auf einen ähnlichen Wertebereich gebracht. Das ist einerseits praktisch, wenn man ihre Einflüsse untereinander vergleichen will. Ist man redoch daran interessiert zu interpretieren, um wie viel man einen Prädiktor erhöhen oder verringern muss, um die abhängige Varable um eine Einheit zu verändern, sind nicht-standardisierte Werte geeigneter.

Wir entscheiden uns für Standardisierung und setzen das Ganze mit `recipe` wie folgt um:

In [ ]:
bikes_recipe <- training(bikes_split) %>%
    recipe(cnt ~.) %>%
    step_corr(all_predictors()) %>% # remove correlating predictors
    step_center(all_predictors(), -all_outcomes()) %>% # centering
    step_scale(all_predictors(), -all_outcomes()) %>% # normalisation
    prep() %>%
    print()

Fertig! Wir haben jetzt unser Rezept für das Modell definiert! Das Tolle an `recipe` ist, dass wir dafür den eigentlichen Datensatz `bikes` gar nicht verändern mussten! `recipe` merkt sich einfach, welche Operationen wir auf den Datensatz anwenden wollen und führt das ganze dann nur aus, wenn wir es zum Trainieren oder Testen benötigen. Außerdem könnten wir mit diesem Datensatz jetzt auch verschiedene Modelle trainieren! Wir haben also eine Art Kochrezept für die richtige Vorbereitung der Daten geschrieben.

#### `bake()` and `juice()`

Wenn wir die Trainings- und Testdaten jetzt als eigenen Datensatz festlegen wollen, müssen wir das Rezept anwenden! Um Trainingsdaten zu erstellen, benutzen wir die Funktion `juice()`. Diese nimmt sich das Rezept und wendet es auf die Trainingsdaten an:

In [ ]:
bikes_training <- bikes_recipe %>%
    juice() %>%
    glimpse()

Wir können sehen dass alle Prädiktoren auf einen Bereich um 0 herum normalisiert worden sind. Das macht es für jetzt umso schwieriger zu lesen, da wir nicht mehr genau sagen können, was die Daten eigentlich bedeuten. Für unser Modell wird es jetzt aber einfacher, gute Vorhersagen zu treffen!

Analog können wir mithilfe von `bake()` die Testdaten ebenfalls vorbereiten:

In [ ]:
bikes_testing <- bikes_recipe %>%
    bake(testing(bikes_split)) %>%
    glimpse()

#### Modellauswahl

Jetzt kommen wir endlich zum eigentlichen Modellieren! Wie letzte Woche können wir uns wieder ein Modell zusammenbasteln. Wir benutzen dafür wieder `decision_tree()` mit der Engine `rpart`, wollen diesmal den Modus aber auf `regression` statt auf `classification` setzen. Als Trainingsdaten benutzen wir die zuvor standardisierten Daten `bikes_training`.

In [ ]:
bikes_model <- decision_tree() %>%
    set_engine("rpart") %>%
    set_mode("regression") %>%
    fit(cnt ~ ., data = bikes_training)

Mithilfe von `rpart.rules()` können wir uns die Regeln unseres trainierten Decision Trees ausgeben lassen:

In [ ]:
bikes_model$fit %>%
    rpart.rules(roundint = FALSE) %>%
    print()

<div style="background-color: #efe3fd"><h1>Präsenzteil</h1></div> 

## Bike sharing data

### Modell evaluieren

<img src="https://parsnip.tidymodels.org/logo.png" alt="parsnip" width="100" align="right"/> Wie zuvor können wir die Funktion `predict()` aus dem Paket `parsnip` verwenden, um Ergebnisse für den Testdatensatz vorherzusagen. Mithilfe von `bind_cols()` verbinden wir dann die Testdaten mit den vorhergesagten Werten.

<img src="https://yardstick.tidymodels.org/logo.png" alt="parsnip" width="100" align="right"/> Die Evaluation erfolgt dann wieder mithilfe des Pakets `yardstick`. Da wir diesmal eine Regressionsanalyse vorgenommen haben statt einem Klassifizierungsproblem, brauchen wir andere Metriken! Wie schon weiter oben können wir wieder einen Satz an Metriken definieren, diesmal interessiert uns aber vor allem R-squared und nicht mehr die Genauigkeit des Modells.

5. Was bedeuten die Metriken `rmse` und `mae`?

In [ ]:
metrics <- metric_set(rsq, rmse, mae)

bikes_model %>%
    predict(bikes_testing) %>%
    bind_cols(bikes_testing) %>%
    metrics(truth = cnt, estimate = .pred)

Unser Modell erreicht ein R-squared von etwa 0.7, was gar nicht so schlecht ist! Es Bedeutet, dass 70 Prozent der Varianz in der abhängigen Variablen durch die unabhängigen erklärt werden kann. Das lässt uns hoffen, dass man auf basis der beobachteten Prädiktorvariablen eine gute Vorhersage treffen kann, wie viele Bikes ausgeliehen sein werden.

#### Variable importance

Um den Einfluss der einzelnen Prädiktoren abzuschätzen, können wir wieder die **Variable Importance** berechnen.

In [ ]:
bikes_model %>% vi(scale = TRUE) %>% # scale the most important variable to 100
    head(6)

Den größten Einfluss hat die Variable `hr`, welche für die Tagesuhrzeit steht. Auch ziemlich entscheidend ist die Temperatur `temp`. Das lässt sich beides relativ klar begründen – nachts fahren vermutlich weniger Menschen mit Bike Sharing als tagsüber, ebenso fahren mehr Menschen wenn es warm genug ist, und weniger Menschen, wenn es draußen kalt ist. In einem richtigen Anwendungsfall ist es sehr wichtig zu verstehen, welche Prädiktoren welchen Einfluss auf die Zielvariable haben.

Um das besser nachzuvollziehen, kann man auch an dieser Stelle nochmal versuchen, die einzelnen Variablen zu plotten. Z.B. können wir den durchschnittlichen Count an ausgeliehen Fahrrädern `cnt` pro Stunde darstellen:

In [ ]:
bikes %>%
    select(hr, cnt) %>%
    group_by(hr) %>%
    summarise(cnt = mean(cnt)) %>%
    ggplot(aes(x = hr, y = cnt)) +
    geom_line() +
    theme_minimal(base_size = 20)

Hier sehen wir bereits, dass die Tageszeit einen starken Einfluss auf `cnt` hat. Nachts gibt es einen Zeitraum, an dem fast gar keine Fahrräder ausgeliehen sind, außerdem gibt es zwei Peaks zu den Stoßzeiten 8 und 17 Uhr, was vermutlich auf Fahrten zwischen Wohnung und Arbeitsstelle zurückzuführen ist.

Auch für die Temperatur können wir die Abhängigkeit von `cnt` zur Temperatur `temp` darstellen und dadurch die Abhängigkeit der Zielvariable `cnt` zum Prädiktor `temp` besser nachvollziehen.

In [ ]:
bikes %>%
    select(temp, cnt) %>%
    group_by(temp) %>%
    summarise(cnt = mean(cnt)) %>%
    ggplot(aes(x = temp, y = cnt)) +
    geom_line() +
    theme_minimal(base_size = 20)

### Vorhersagen treffen

Wie bei den Pilzen müssen wir jetzt einen neuen `tibble` erstellen, den wir anschließend mithilfe von `predict()` vorhersagen können. Dabei ist zu beachten, dass einige der Prädiktorvariablen vorher bereits im Datensatz normalisiert worden waren! Das ist im Readme hinterlegt und im folgenden Code auch nochmal kommentiert.

In [ ]:
vorhersage <- tibble(
    season = 4, # 1 spring, 2 summer, 3 fall, 4 winter
    yr = 0, # 0 for 2011, 1 for 2012
    mnth = 2, # 2 for February
    hr = 11, # 11:00
    holiday = 0, # 0 no holiday, 1 is holiday
    weekday = 5, # weekday
    workingday = 1, # 0 no working day, 1 week day
    weathersit = 2, 
    # weathersit
    # 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    # 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    # 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    # 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
    
    temp = 30 / 41, # Celsius, normalised temperature through division by 41
    atemp = 25 / 50, # Celsius, normalised feeling temperature through division by 50
    hum = 20 / 100, # percentage, normalised through division by 100
    windspeed = 35 / 67, # normalised through division 67
)

Anschließend müssen wir die Daten wieder standardisieren, damit sie zu den Trainingsdaten passen! Das machen wir wieder mithilfe von `bake()`.

In [ ]:
vorhersage_standardised <- bikes_recipe %>%
    bake(vorhersage) %>%
    glimpse()

Nun können wir die Vorhersage mit `predict()` treffen:

In [ ]:
bikes_model %>%
    predict(vorhersage_standardised) 

Wir könnten jetzt also für einen beliebigen Zeitpunkt die Anzahl der ausgeliehen Fahrräder vorhersagen, sofern uns die notwendigen Wetterdaten zur Verfügung stehen!

### Modell verbessern

Für die Modellverbesserung gibt es neben den bisher kennengelernten Kniffen wie einer größeren Datenbasis auch die Möglichkeit, ein verbessertes Modell zu wählen. Die Modelle, die wir bisher verwendet haben, sind für erstmal ziemliche Blackboxen, sofern wir nicht direkt den Quellcode einsehen oder uns tief in die Dokumentation einlesen. Daher bleibt uns am ehesten übrig, nach weiteren R-Paketen zu suchen, die eine verbesserte Regressionsanalyse mit Decision Trees umsetzen. Ein Beispiel hierfür ist das Paket [Cubist](https://cran.r-project.org/web/packages/Cubist/index.html). Dieses basiert auf Quinlan's M5 model und ist ein typisches Beispiel, bei dem eine Funktion aufbauend auf einem Paper umgesetzt wurde. 

> Quinlan, J. R. (1992, November). Learning with continuous classes. In 5th Australian joint conference on artificial intelligence (Vol. 92, pp. 343-348).

Um den Algorithmus besser zu verstehen, wäre dann der nächste Schritt, das Paper zu lesen. Dafür haben wir natürlich gerade keine Zeit. Tendenziell ist es aber wünschenswert, dass man ein Grundverständnis von den Verfahren hat, die man anwendet. Spannend ist auch noch das Jahr, in dem das Paper veröffentlich wurde (1992). Das ist schon über 30 Jahre her, und war damals state-of-the-art Machine Learning. Bis ChatGPT ist es also noch ein weiter Weg...

## Zweites Anwendungsbeispiel

Und nun zu einem komplett neuen Thema: Es geht jetzt um "Abalone", auch Seeohren genannt. Das ist eine Meeresschnecken-Art. Mehr zu diesen faszinierenden Tieren findet ihr auf [Wikipedia](https://de.wikipedia.org/wiki/Seeohren). Die Aufgabe, der wir uns nun stellen, ist die Bestimmung des Alter einzelner Exemplare der Gattung. Hier ein paar Infos zum Forschungszusammenhang und Datensatz:

> Predicting the age of abalone from physical measurements.  The age of abalone is determined by cutting the shell through the cone, staining it, and counting the number of rings through a microscope -- a boring and time-consuming task.  Other measurements, which are easier to obtain, are used to predict the age.  Further information, such as weather patterns and location (hence food availability) may be required to solve the problem.
>
> From the original data examples with missing values were removed (the majority having the predicted value missing), and the ranges of the continuous values have been scaled for use with an ANN (by dividing by 200).

[Abalone data](http://archive.ics.uci.edu/dataset/1/abalone)

> Nash,W., Sellers,T., Talbot,S., Cawthorn,A., and Ford,W. (1995). Abalone. UCI Machine Learning Repository. https://doi.org/10.24432/C55C7W.

6. Schaut im Readme unter `"data/abalone/abalone.names"` die Bedeutung der verschiedenen Variablen des Datensatzes nach.

7. Ladet den Abalone-Datensatz aus dem Pfad `"data/abalone/abalone.data"` in einen neuen Dataframe und exploriert die verschiedenen Variablen des Datensatzes.  In Bezug auf das Zitat weiter oben aus der ursprünglichen Veröffentlichung des Datensatzes - welche Variable stellt eine interessante Zielvariable dar?

8. Stellt die verschiedenen Variablen mithilfe von `ggplot2` und `facet_wrap()` dar. Wie sind die Variablen verteilt?

9. Teilt den Datensatz mithilfe von `initial_split()` in Trainings- und Testdaten auf. Achtung: ihr dürft lediglich numerische Variablen verwenden – eventuell müsst ihr einige Variablen vorher herausfiltern.

10. Erstellt mithilfe von `recipe()` ein Rezept für die Trainingsdaten. Findet mithilfe von `step_corr()` heraus, welche Variablen miteinander korrelieren. Diskutiert, warum eine Korrelation zwischen den Variablen naheliegend ist.

11. Im nächsten Schritt müsst ihr `bake()` und `juice()` verwenden und das vorher erstelle `recipe` auf die Trainings- und Testdaten anwenden.

12. Erstellt ein Modell vom Typ `decision_tree()` mit der Engine `"rpart"` und dem Modus `"regression"` und trainiert das Modell mit den Trainingsdaten `abalone_training`. Schaut euch anscheinend die Regeln des Modells an und überlegt kurz, welche Prädiktoren wohl eine große Rolle spielen.

13. Berechnet mithilfe der Testdaten `abalone_testing` sowie `predict()` und `metrics()` den R-squared value des Modells. Fragt anschließend ChatGPT oder Google, wie ihr das Ergebnis interpretieren würdet. Ist das Ergebnis gut genug für den Anwendungsfall? Inwiefern hängt die Beantwortung dieser Frage vom Kontext der Fragestellung ab (also der Fachdisziplin)?

14. Was für ein Alter (Anzahl der Ringe + 1.5) würdet ihr für eine Abalone mit `height = 0.85`, `weight_shucked = 0.6` und `weight_shell = 0.8` vorhersagen? Vergesst nicht, die neuen Daten vorher mit `bake()` zu standardisieren. Was ist der Mean Absolute Error `mae` für das Ergebnis?

## Guess the Correlation

Als Abschluss der Übung wollen wir noch ein kleines Spiel zusammen spielen, und zwar [GeoGebra Guess the Correlation](https://www.geogebra.org/m/gvanchqz). Klickt auf die Webseite des Spiels und fangt an, das Spiel zusammen zu spielen. Das Spiel ist sehr simpel und relativ selbsterklärend, es geht darum, den R-squared Wert einer Verteilung zu erraten. Dadurch bekommt ihr ein Gefühl für die Güte von linearen Modellen.

## Zusammenfassung

In dieser Übung haben wir die Regressionsanalyse kennengelernt. Regressionen sind eines der am häufigsten verwendeten Tools in der Statistik. Sie werden euch immer wieder begegnen. Die Umsetzung in R kann sowohl mit `base`, innerhalb von Grafiken mit `ggplot2` als auch mit `tidymodels` erfolgen, und wir können die Regressionsanalyse auch mit weiteren ML-Modellen wie z.B. Decision Trees verknüpfen.

Außerdem haben wir in dieser Übung den Modellierungsprozess etwas formalisiert, und zwar mithilfe des Pakets `recipe`. Dieses Paket hilft uns dabei, stringente und gleichartige Abläufe für unsere Modelle zu bauen.

In der nächsten Sitzung wird es um den **Naive Bayes Classifier** gehen. Dabei werden wir nochmal auf `recipe` verzichten, und einige Funktionen des Modells selber schreiben!